<a href="https://colab.research.google.com/github/sourabhsdx/waylyf/blob/master/exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math 
from nltk.corpus import wordnet
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
def read_data(variable):
      variable = variable
      #print(type(variable))
      #tokenized_sent = word_tokenize(variable)
      tokenizer = RegexpTokenizer(r'\w+')
      tokenized_sent = tokenizer.tokenize(variable)
      stop_words=set(stopwords.words("english"))
      filtered_stem_sent=[]
      porter = PorterStemmer()
      lemmatizer = WordNetLemmatizer()
      for w in tokenized_sent:
        w = w.lower()
        if w not in stop_words:
          #new_w = porter.stem(w)
          new_w = lemmatizer.lemmatize(w,pos ="a")
          if new_w == w:
            new_w = lemmatizer.lemmatize(w,pos ="n")
            if new_w == w:
              new_w = lemmatizer.lemmatize(w,pos ="v")
              if new_w == w:
                new_w = lemmatizer.lemmatize(w,pos ="r")
          filtered_stem_sent.append(new_w)
      return (filtered_stem_sent)


def _create_frequency_matrix(sentences, common_ans):
    frequency_matrix = {}
    freq_table = []
    result_list = list(set(common_ans) - set(sentences))
    for word in sentences:
      if word in freq_table:
        frequency_matrix[word] = frequency_matrix[word]+1
      else:
        frequency_matrix[word] = 1
        freq_table.append(word)
      for w in result_list:
        synonyms = []
        for syn in wordnet.synsets(w):
          for l in syn.lemmas():
            synonyms.append(l.name())
        set_syn = set(synonyms)
        frequency_matrix[w] = 0
        for w1 in set_syn:
          if w1 in freq_table:
            frequency_matrix[w] = 1
            freq_table.append(word)
            break

    return(frequency_matrix)

def _create_tf_matrix(frequency_matrix, T_length):
  TF = {}
  for words in frequency_matrix:
    TF[words] = frequency_matrix[words]/T_length
  return (TF)

def _create_frequency_matrix_for_idf(common_ans, aut_ans, stu_ans):
    idf_matrix = {}
    for words in common_ans:
      if words in aut_ans:
        if words in stu_ans:
          idf_matrix[words] = 2
        else:
          idf_matrix[words] = 1
      else:
        idf_matrix[words] = 1
    return idf_matrix

def _create_idf(idf_tb):
  idf = {}
  for words in idf_tb:
    idf[words] = math.log(1+(2/(idf_tb[words])), 10) 
  return idf
def _create_weight(tf,idf):
  weight = {}
  for i in tf:
    weight[i] = tf[i]*idf[i]
  return weight

author_ans = "I am a good boy "
student_ans = "i am a soun boy"
from sklearn.metrics.pairwise import cosine_similarity
def main(author_ans,student_ans):
  aut_ans = read_data(author_ans)
  stu_ans = read_data(student_ans)
  common_ans = []
  for i in aut_ans:
    if i not in common_ans:
      common_ans.append(i)
  for i in stu_ans:
    if i not in common_ans:
      common_ans.append(i)
  aut_frq_tb = _create_frequency_matrix(aut_ans, common_ans)
  stu_frq_tb = _create_frequency_matrix(stu_ans, common_ans)
  #length = len(aut_ans)
  # print("frq table", aut_frq_tb)
  # print("frq table",stu_frq_tb)
  aut_tf = _create_tf_matrix(aut_frq_tb,len(aut_ans))
  stu_tf = _create_tf_matrix(stu_frq_tb,len(stu_ans))
  # print(aut_ans)
  # print(stu_ans)
  # print(common_ans)
  idf_tb = _create_frequency_matrix_for_idf(common_ans, aut_ans, stu_ans)
  #print(idf_tb)
  idf = _create_idf(idf_tb)
  aut_weight = _create_weight(aut_tf, idf)
  stu_weight = _create_weight(stu_tf, idf)
  # print("tf of author : ",aut_tf)
  # print("tf of student : ",stu_tf)
  # print("idf : ",idf)
  # print(aut_weight)
  # print(stu_weight)
  x = []
  y = []
  for words in common_ans:
    #print(words,aut_weight[words],words,stu_weight[words])
    x.append(aut_weight[words])
    y.append(stu_weight[words])
  x1 = np.array([x])
  y1 = np.array([y])
  print(cosine_similarity(x1,y1))
main(author_ans,student_ans)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[[0.28472944]]
